# ROOT in Python tutorial for MicroBooNE

## Importing packages

NumPy: Numerical calculations, handling arrays efficiently

In [8]:
import numpy as np

Pandas: Dataframe wrapper of numpy with loads of tricks and extras

In [9]:
import pandas as pd
pd.set_option('display.max_columns', 500)

Graphical toold: matplotlib and seaborn 

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

uproot: ROOT I/O in pure Python and Numpy.

[GitHub](https://github.com/scikit-hep/uproot)  

Tutorials:

[Binder uproot tutorial](https://mybinder.org/v2/gh/scikit-hep/uproot/master?filepath=binder%2Ftutorial.ipynb)

https://github.com/scikit-hep/awkward-array/blob/master/binder/jagged-arrays.ipynb

https://github.com/scikit-hep/uproot/blob/master/binder/version-3-features.ipynb

In [12]:
import uproot

## Constants

In [13]:
lower = [-1.55, -115.53, 0.1]
upper = [254.8, 117.47, 1036.9]
fidvol_arr = [10,10,10,10,10,50]

file_name = 'flashID_nue_mcc9_nov_50k.root'

## Exploring the ROOT file and loading it

In [15]:
file = uproot.open(file_name)

In [16]:
file

<ROOTDirectory b'flashID_nue_mcc9_nov_50k.root' at 0x7fac037cd4a8>

In [17]:
file.keys()

[b'nueCCAnalyser;1',
 b'pandoraFlashEventBuilding;1',
 b'FRH;1',
 b'FRV;1',
 b'rICKR;1',
 b'rICKI;1',
 b'PreC;1',
 b'PostC;1',
 b'PostO;1',
 b'PreD;1',
 b'PostDO;1',
 b'ER;1']